![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/3.2.Sentence_Entity_Resolvers_with_EntityChunkEmbeddings.ipynb)

# Sentence Entity Resolvers with EntityChunkEmbeddings

## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

## Colab Setup

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

## Starting Spark

In [ ]:
import json
import os

from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import sparknlp_jsl
import sparknlp

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.3.0
Spark NLP_JSL Version : 5.3.0


## Creating pipeline

In [ ]:
documenter = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols("document") \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols("sentence") \
    .setOutputCol("token")

embeddings = WordEmbeddingsModel() \
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

posology_ner_model = MedicalNerModel()\
    .pretrained("ner_posology_large", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols("sentence", "token", "ner")\
    .setOutputCol("ner_chunk")

pos_tager = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token")\
    .setOutputCol("pos_tag")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tag", "token"])\
    .setOutputCol("dependencies")

entity_chunk_embeddings = EntityChunkEmbeddings()\
    .pretrained("sbiobert_base_cased_mli","en","clinical/models")\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("drug_chunk_embeddings")\
    .setMaxSyntacticDistance(3)

entity_chunk_embeddings.setTargetEntities({"DRUG": ["STRENGTH", "ROUTE", "FORM"]})
entity_chunk_embeddings.setEntityWeights({"DRUG": 0.8, "STRENGTH": 0.2, "ROUTE": 0.2, "FORM": 0.2})

rxnorm_re = SentenceEntityResolverModel\
    .pretrained("sbiobertresolve_rxnorm_augmented_re", "en","clinical/models")\
    .setInputCols(["drug_chunk_embeddings"])\
    .setOutputCol("rxnorm_code")\
    .setDistanceFunction("EUCLIDEAN")

rxnorm_pipeline_re = Pipeline(
    stages = [
        documenter,
        sentence_detector,
        tokenizer,
        embeddings,
        posology_ner_model,
        ner_converter,
        pos_tager,
        dependency_parser,
        entity_chunk_embeddings,
        rxnorm_re
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology_large download started this may take some time.
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
[OK!]
sbiobertresolve_rxnorm_augmented_re download started this may take some time.
[OK!]


### Sample Text

In [ ]:
import pandas as pd
sampleText = ["The patient was given metformin 500 mg tablet, 2.5 mg of coumadin and then ibuprofen.",
              "The patient was given metformin 400 mg, coumadin 5 mg, coumadin, amlodipine 10 MG tablet"]

sample_df = pd.DataFrame({'text':sampleText}).reset_index()

In [ ]:
data_df = spark.createDataFrame(sample_df)

results = rxnorm_pipeline_re.fit(data_df).transform(data_df)

## Chunks extracted by NER model

In [ ]:
results.select("index", F.explode(F.arrays_zip(results.ner_chunk.result,
                                               results.ner_chunk.metadata)).alias("cols")) \
.select("index", F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----+----------+---------+
|index|chunk     |ner_label|
+-----+----------+---------+
|0    |metformin |DRUG     |
|0    |500 mg    |STRENGTH |
|0    |tablet    |FORM     |
|0    |2.5 mg    |STRENGTH |
|0    |coumadin  |DRUG     |
|0    |ibuprofen |DRUG     |
|1    |metformin |DRUG     |
|1    |400 mg    |STRENGTH |
|1    |coumadin  |DRUG     |
|1    |5 mg      |STRENGTH |
|1    |coumadin  |DRUG     |
|1    |amlodipine|DRUG     |
|1    |10 MG     |STRENGTH |
|1    |tablet    |FORM     |
+-----+----------+---------+



## Merged chunks by internal relation extraction model feature

- We specified the relations as following by `.setTargetEntities` parameter in the EntityChunkEmbeddings annotator :    
`.setTargetEntities({"DRUG": ["STRENGTH", "ROUTE", "FORM"]})`

- EntityChunkEmbeddings calculates those new chunks embeddings according to the weights specified in`.setEntityWeights` parameter.

`.setEntityWeights({"DRUG": 0.8, "STRENGTH": 0.2, "ROUTE": 0.2, "FORM": 0.2})`

In [ ]:
results.select('index','drug_chunk_embeddings.result').show(truncate = False)

+-----+--------------------------------------------------------------------+
|index|result                                                              |
+-----+--------------------------------------------------------------------+
|0    |[metformin 500 mg tablet, 2.5 mg coumadin, ibuprofen]               |
|1    |[metformin 400 mg, coumadin 5 mg, coumadin, amlodipine 10 MG tablet]|
+-----+--------------------------------------------------------------------+



## RxNorm Results

In [ ]:
results.select('index', F.explode(F.arrays_zip(results.drug_chunk_embeddings.result,
                                               results.rxnorm_code.result,
                                               results.rxnorm_code.metadata).alias("col")))\
        .select('index', F.expr("col['0']").alias("chunk"),
                         F.expr("col['1']").alias("rxnorm_code_"),
                         F.expr("col['2']['resolved_text']").alias("Concept_Name")).show(truncate = 50)

+-----+-----------------------+------------+---------------------------------+
|index|                  chunk|rxnorm_code_|                     Concept_Name|
+-----+-----------------------+------------+---------------------------------+
|    0|metformin 500 mg tablet|      860974|   metformin hydrochloride 500 MG|
|    0|        2.5 mg coumadin|      855313|warfarin sodium 2.5 MG [Coumadin]|
|    0|              ibuprofen|     1747293|              ibuprofen Injection|
|    1|       metformin 400 mg|      332809|                 metformin 400 MG|
|    1|          coumadin 5 mg|      855333|  warfarin sodium 5 MG [Coumadin]|
|    1|               coumadin|      202421|                         Coumadin|
|    1|amlodipine 10 MG tablet|      308135|     amlodipine 10 MG Oral Tablet|
+-----+-----------------------+------------+---------------------------------+

